In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm import tqdm

def subtract_images_in_folder(folder_path, output_folder_path):
    # Get list of image files in the folder
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])

    # Initialize the progress bar
    progress_bar = tqdm(total=len(image_files) - 9, desc="Processing images")

    for i in range(len(image_files) - 9):
        images = [cv2.imread(os.path.join(folder_path, image_files[i + j])) for j in range(10)]
        output_path = os.path.join(output_folder_path, f'subtracted_{i}.png')

        # Check if images have the same size
        if any(image.shape != images[0].shape for image in images):
            raise ValueError(f"All images must have the same dimensions for subtraction.")

        # Subtract the images
        subtracted_image = cv2.subtract(images[0], images[1])
        for j in range(1, 9):
            subtracted_image = cv2.add(subtracted_image, cv2.subtract(images[j], images[j + 1]))

        # Denoise the image
        denoised_image = cv2.fastNlMeansDenoisingColored(subtracted_image, None, h=10, templateWindowSize=7, searchWindowSize=21)

        # Set contrast to 100
        contrast_image = cv2.convertScaleAbs(denoised_image, alpha=3.0, beta=0)

        # Denoise the image
        #denoised_image2 = cv2.fastNlMeansDenoisingColored(contrast_image, None, h=10, templateWindowSize=7, searchWindowSize=21)

        # Save the result
        cv2.imwrite(output_path, contrast_image)

        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

# Example usage
subtract_images_in_folder("Captures/0", "Captures/Out/contrast")

Processing images:   0%|          | 0/18019 [03:22<?, ?it/s]


KeyboardInterrupt: 